In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle_2.json to kaggle_2.json


{'kaggle_2.json': b'{"username":"pushkum","key":"8b0383a1515fe4ec355bc96ceb789cbb"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d aryashah2k/indian-medicinal-leaves-dataset

 84% 7.52G/9.00G [01:21<00:10, 146MB/s]

In [ ]:
import zipfile
with zipfile.ZipFile('indian-medicinal-leaves-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('indian-medicinal-leaves-dataset')

In [ ]:
import shutil

# Path to the folder you want to copy
source_folder = '/content/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset'

# Path to the destination folder where you want to copy the source folder
destination_folder = '/content/Medicinal Leaf dataset'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

'/content/Medicinal Leaf dataset'

In [ ]:
!rm -rf /content/indian-medicinal-leaves-dataset.zip

In [ ]:
import shutil
import os

# Directory to be zipped
directory_to_zip = '/content/Medicinal Leaf dataset'

# Name of the zip file
zip_file_name = 'Medicinal_Leaf_Dataset.zip'

# Create a zip file
shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', directory_to_zip)

# Optionally, remove the original directory
shutil.rmtree(directory_to_zip)

# Move the zip file to the desired location
shutil.move(zip_file_name, '/content/drive/MyDrive')

'/content/drive/MyDrive/Medicinal_Leaf_Dataset.zip'

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from bs4 import BeautifulSoup
import requests

# Load the data
data_dir = '/path/to/your/dataset'
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Use ResNet50 as the base model
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(train_ds.class_names), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

# Function to scrape medicinal benefits from Google
def get_medicinal_benefits(plant_name):
    url = f"https://www.google.com/search?q={plant_name}+medicinal+benefits"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    benefits = ' '.join([desc.text for desc in soup.find_all('div')])
    return benefits[:500]  # Limit the benefits to 500 characters

# Test the model and print the medicinal benefits
for images, labels in val_ds.take(1):
    preds = model.predict(images)
    for i in range(len(preds)):
        plant_name = train_ds.class_names[preds[i].argmax()]
        print(f"Image {i+1}:")
        print(f"Predicted plant: {plant_name}")
        print(f"Medicinal benefits: {get_medicinal_benefits(plant_name)}\n")
